In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk
import re

%matplotlib inline

In [2]:
df = pd.read_csv('../input/twitter-sentimental-analysis/0000000000002747_training_twitter_x_y_train.csv')
df

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   tweet_id                10980 non-null  int64 
 1   airline_sentiment       10980 non-null  object
 2   airline                 10980 non-null  object
 3   airline_sentiment_gold  31 non-null     object
 4   name                    10980 non-null  object
 5   negativereason_gold     24 non-null     object
 6   retweet_count           10980 non-null  int64 
 7   text                    10980 non-null  object
 8   tweet_coord             776 non-null    object
 9   tweet_created           10980 non-null  object
 10  tweet_location          7430 non-null   object
 11  user_timezone           7403 non-null   object
dtypes: int64(2), object(10)
memory usage: 1.0+ MB


In [4]:
df.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','user_timezone','tweet_location','tweet_created'],axis=1,inplace=True)
df

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...
...,...,...
10975,neutral,@AmericanAir followback
10976,positive,@united thanks for the help. Wish the phone re...
10977,negative,@usairways the. Worst. Ever. #dca #customerser...
10978,negative,@nrhodes85: look! Another apology. DO NOT FLY ...


In [5]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()
df['airline_sentiment'] = number.fit_transform(df['airline_sentiment'].astype('str'))
df

,airline_sentiment,text
0,0,"@SouthwestAir I am scheduled for the morning, ..."
1,2,@SouthwestAir seeing your workers time in and ...
2,2,@united Flew ORD to Miami and back and had gr...
3,0,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,0,@united so our flight into ORD was delayed bec...
...,...,...
10975,1,@AmericanAir followback
10976,2,@united thanks for the help. Wish the phone re...
10977,0,@usairways the. Worst. Ever. #dca #customerser...
10978,0,@nrhodes85: look! Another apology. DO NOT FLY ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  10980 non-null  int64 
 1   text               10980 non-null  object
dtypes: int64(1), object(1)
memory usage: 171.7+ KB


In [7]:
# removes pattern in the input text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt

In [8]:
df['clean_text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")
df

,airline_sentiment,text,clean_text
0,0,"@SouthwestAir I am scheduled for the morning, ...","I am scheduled for the morning, 2 days after ..."
1,2,@SouthwestAir seeing your workers time in and ...,seeing your workers time in and time out goin...
2,2,@united Flew ORD to Miami and back and had gr...,Flew ORD to Miami and back and had great cre...
3,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,that's horse radish 😤🐴
4,0,@united so our flight into ORD was delayed bec...,so our flight into ORD was delayed because of...
...,...,...,...
10975,1,@AmericanAir followback,followback
10976,2,@united thanks for the help. Wish the phone re...,thanks for the help. Wish the phone reps coul...
10977,0,@usairways the. Worst. Ever. #dca #customerser...,the. Worst. Ever. #dca #customerservice
10978,0,@nrhodes85: look! Another apology. DO NOT FLY ...,: look! Another apology. DO NOT FLY


In [9]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
stops, string.punctuation

({'!',
  '"',
  '#',
  '$',
  '%',
  '&',
  "'",
  '(',
  ')',
  '*',
  '+',
  ',',
  '-',
  '.',
  '/',
  ':',
  ';',
  '<',
  '=',
  '>',
  '?',
  '@',
  '[',
  '\\',
  ']',
  '^',
  '_',
  '`',
  'a',
  'about',
  'above',
  'after',
  'again',
  'against',
  'ain',
  'all',
  'am',
  'an',
  'and',
  'any',
  'are',
  'aren',
  "aren't",
  'as',
  'at',
  'be',
  'because',
  'been',
  'before',
  'being',
  'below',
  'between',
  'both',
  'but',
  'by',
  'can',
  'couldn',
  "couldn't",
  'd',
  'did',
  'didn',
  "didn't",
  'do',
  'does',
  'doesn',
  "doesn't",
  'doing',
  'don',
  "don't",
  'down',
  'during',
  'each',
  'few',
  'for',
  'from',
  'further',
  'had',
  'hadn',
  "hadn't",
  'has',
  'hasn',
  "hasn't",
  'have',
  'haven',
  "haven't",
  'having',
  'he',
  'her',
  'here',
  'hers',
  'herself',
  'him',
  'himself',
  'his',
  'how',
  'i',
  'if',
  'in',
  'into',
  'is',
  'isn',
  "isn't",
  'it',
  "it's",
  'its',
  'itself',
  'just',
  'll',


In [10]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [11]:
x = df['clean_text']

In [12]:
list(x)

[' I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled',
 ' seeing your workers time in and time out going above and beyond is why I love flying with you guys. Thank you!',
 ' Flew ORD to Miami and back and  had great crew, service on both legs. THANKS',
 "  that's horse radish 😤🐴",
 ' so our flight into ORD was delayed because of Air Force One, but the last flight to SBN is at 8:20, 5 mins from now we just landed.',
 ' Why did you load us in this flying sardine can if you knew the pilots were 2 hours Late Flight?? #incompetent beyond belief',
 " that is a stock response. Delays not as frustrating as poor cust serv &amp; being told by 3 ppl to wait &amp; they'd come back but did not.",
 " That'd be nice! Hoping to rack up enough miles to take a trip to Seattle and enjoy a perfect latte in the city of coffee.",
 ' frankly worse customer service ever. Problems will happen, how you deal defines a company. Never ag

In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
Documents = [word_tokenize(doc) for doc in list(x)]
Documents

[['I',
  'am',
  'scheduled',
  'for',
  'the',
  'morning',
  ',',
  '2',
  'days',
  'after',
  'the',
  'fact',
  ',',
  'yes',
  '..',
  'not',
  'sure',
  'why',
  'my',
  'evening',
  'flight',
  'was',
  'the',
  'only',
  'one',
  'Cancelled',
  'Flightled'],
 ['seeing',
  'your',
  'workers',
  'time',
  'in',
  'and',
  'time',
  'out',
  'going',
  'above',
  'and',
  'beyond',
  'is',
  'why',
  'I',
  'love',
  'flying',
  'with',
  'you',
  'guys',
  '.',
  'Thank',
  'you',
  '!'],
 ['Flew',
  'ORD',
  'to',
  'Miami',
  'and',
  'back',
  'and',
  'had',
  'great',
  'crew',
  ',',
  'service',
  'on',
  'both',
  'legs',
  '.',
  'THANKS'],
 ['that', "'s", 'horse', 'radish', '😤🐴'],
 ['so',
  'our',
  'flight',
  'into',
  'ORD',
  'was',
  'delayed',
  'because',
  'of',
  'Air',
  'Force',
  'One',
  ',',
  'but',
  'the',
  'last',
  'flight',
  'to',
  'SBN',
  'is',
  'at',
  '8:20',
  ',',
  '5',
  'mins',
  'from',
  'now',
  'we',
  'just',
  'landed',
  '.'],
 

In [15]:
all_Documents=[]
#this variable is going to contain all the words from all our tokenized documents.
for doc in Documents:
    list_of = []
    for word in doc:
        if (word.lower() not in stops) and len(word)!=1 and len(word)!=2 and word[0]!="'" and word!="n't" and word[0]!=".":
            #I dont't want to include words with length 1 and 2 in my vocabulary because these words are pretty much useless.
            # and they might either be the stops which are not there in "stops" variable or some punctuation marks which are
            # not there in "punctuations" variable. apart from the words of length 1 and 2 i have also removed some words separately because these words are most probably present in all the documents so they are not helping us much in classification.
            list_of.append(word.lower())
    all_Documents.append(list_of)

In [16]:
all_Documents

[['scheduled',
  'morning',
  'days',
  'fact',
  'yes',
  'sure',
  'evening',
  'flight',
  'one',
  'cancelled',
  'flightled'],
 ['seeing',
  'workers',
  'time',
  'time',
  'going',
  'beyond',
  'love',
  'flying',
  'guys',
  'thank'],
 ['flew',
  'ord',
  'miami',
  'back',
  'great',
  'crew',
  'service',
  'legs',
  'thanks'],
 ['horse', 'radish'],
 ['flight',
  'ord',
  'delayed',
  'air',
  'force',
  'one',
  'last',
  'flight',
  'sbn',
  '8:20',
  'mins',
  'landed'],
 ['load',
  'flying',
  'sardine',
  'knew',
  'pilots',
  'hours',
  'late',
  'flight',
  'incompetent',
  'beyond',
  'belief'],
 ['stock',
  'response',
  'delays',
  'frustrating',
  'poor',
  'cust',
  'serv',
  'amp',
  'told',
  'ppl',
  'wait',
  'amp',
  'come',
  'back'],
 ['nice',
  'hoping',
  'rack',
  'enough',
  'miles',
  'take',
  'trip',
  'seattle',
  'enjoy',
  'perfect',
  'latte',
  'city',
  'coffee'],
 ['frankly',
  'worse',
  'customer',
  'service',
  'ever',
  'problems',
  'ha

In [17]:
sentiment = list(df['airline_sentiment'])

In [18]:
sentiment

[0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,


In [19]:
Documents = []
for i in range(10980):
    Documents.append(([stemmer.stem(x) for x in all_Documents[i]], sentiment[i]))
Documents

[(['schedul',
   'morn',
   'day',
   'fact',
   'ye',
   'sure',
   'even',
   'flight',
   'one',
   'cancel',
   'flightl'],
  0),
 (['see',
   'worker',
   'time',
   'time',
   'go',
   'beyond',
   'love',
   'fli',
   'guy',
   'thank'],
  2),
 (['flew', 'ord', 'miami', 'back', 'great', 'crew', 'servic', 'leg', 'thank'],
  2),
 (['hors', 'radish'], 0),
 (['flight',
   'ord',
   'delay',
   'air',
   'forc',
   'one',
   'last',
   'flight',
   'sbn',
   '8:20',
   'min',
   'land'],
  0),
 (['load',
   'fli',
   'sardin',
   'knew',
   'pilot',
   'hour',
   'late',
   'flight',
   'incompet',
   'beyond',
   'belief'],
  0),
 (['stock',
   'respons',
   'delay',
   'frustrat',
   'poor',
   'cust',
   'serv',
   'amp',
   'told',
   'ppl',
   'wait',
   'amp',
   'come',
   'back'],
  0),
 (['nice',
   'hope',
   'rack',
   'enough',
   'mile',
   'take',
   'trip',
   'seattl',
   'enjoy',
   'perfect',
   'latt',
   'citi',
   'coffe'],
  2),
 (['frankli',
   'wors',
   'cust

In [20]:
all_words = []
for doc in Documents:
    all_words += doc[0]
all_words

['schedul',
 'morn',
 'day',
 'fact',
 'ye',
 'sure',
 'even',
 'flight',
 'one',
 'cancel',
 'flightl',
 'see',
 'worker',
 'time',
 'time',
 'go',
 'beyond',
 'love',
 'fli',
 'guy',
 'thank',
 'flew',
 'ord',
 'miami',
 'back',
 'great',
 'crew',
 'servic',
 'leg',
 'thank',
 'hors',
 'radish',
 'flight',
 'ord',
 'delay',
 'air',
 'forc',
 'one',
 'last',
 'flight',
 'sbn',
 '8:20',
 'min',
 'land',
 'load',
 'fli',
 'sardin',
 'knew',
 'pilot',
 'hour',
 'late',
 'flight',
 'incompet',
 'beyond',
 'belief',
 'stock',
 'respons',
 'delay',
 'frustrat',
 'poor',
 'cust',
 'serv',
 'amp',
 'told',
 'ppl',
 'wait',
 'amp',
 'come',
 'back',
 'nice',
 'hope',
 'rack',
 'enough',
 'mile',
 'take',
 'trip',
 'seattl',
 'enjoy',
 'perfect',
 'latt',
 'citi',
 'coffe',
 'frankli',
 'wors',
 'custom',
 'servic',
 'ever',
 'problem',
 'happen',
 'deal',
 'defin',
 'compani',
 'never',
 'unit',
 'yeah',
 'haha',
 'never',
 'one',
 'expens',
 'much',
 'fun',
 'destinationdragon',
 'mco-',
 'dc

In [21]:
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)
features = [i[0] for i in common]

In [22]:
len(features)

3000

In [23]:
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [24]:
categories = [category for document, category in Documents]
categories

[0,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,


In [25]:
text_documents = [" ".join(document) for document, category in Documents]
text_documents

['schedul morn day fact ye sure even flight one cancel flightl',
 'see worker time time go beyond love fli guy thank',
 'flew ord miami back great crew servic leg thank',
 'hors radish',
 'flight ord delay air forc one last flight sbn 8:20 min land',
 'load fli sardin knew pilot hour late flight incompet beyond belief',
 'stock respons delay frustrat poor cust serv amp told ppl wait amp come back',
 'nice hope rack enough mile take trip seattl enjoy perfect latt citi coffe',
 'frankli wors custom servic ever problem happen deal defin compani never unit',
 'yeah haha never one expens much fun destinationdragon',
 'mco- dca flight almost full peopl screw msy-dca cancel flightat cancel flight swa=mistak',
 'easiest way get ticket receipt get one check get one onlin thank',
 'love chang loung chees veggi oliv addit cracker snack mix',
 'receiv bad custom servic end spend sever hundr dollar accommod famili cxl flight',
 'kill sche flight board min mine arriv gate side airport geniou',
 'wee

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
count_vec = CountVectorizer(max_features = 2000, ngram_range=(1,2))
x_train_features = count_vec.fit_transform(text_documents)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [28]:
count_vec.get_feature_names()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '000 mile',
 '10',
 '100',
 '11',
 '12',
 '130',
 '15',
 '150',
 '17',
 '19',
 '1hr',
 '1st',
 '1st class',
 '20',
 '200',
 '2015',
 '22',
 '23',
 '24',
 '24hr',
 '25',
 '28',
 '2day',
 '2hr',
 '2nd',
 '2nd time',
 '30',
 '300',
 '30am',
 '30min',
 '30pm',
 '3hr',
 '3rd',
 '40',
 '400',
 '45',
 '45min',
 '4th',
 '50',
 '500',
 '5hr',
 '700',
 '728',
 '737',
 '777',
 '7am',
 '800',
 '800 number',
 'a320',
 'abl',
 'abl get',
 'absolut',
 'absurd',
 'accept',
 'access',
 'accommod',
 'account',
 'acct',
 'act',
 'action',
 'actual',
 'ad',
 'add',
 'addit',
 'address',
 'admir',
 'admir club',
 'advanc',
 'advantag',
 'advis',
 'advisori',
 'affect',
 'afford',
 'afternoon',
 'age',
 'agent',
 'agent help',
 'agent phone',
 'agent said',
 'ago',
 'ahead',
 'air',
 'aircraft',
 'airfar',
 'airlin',
 'airlin cancel',
 'airlin ever',
 'airplan',
 'airport',
 'airport hour',
 'airway',
 'alert',
 'all',
 'allianc',
 'allow',
 'almost',
 'almost hour',
 'alon',
 'along',
 'alr

In [29]:
from sklearn.svm import SVC

In [30]:
svc = SVC()
svc.fit(x_train_features, categories)

SVC()

In [31]:
svc.score(x_train_features, categories)

0.8981785063752277

In [32]:
dft = pd.read_csv('../input/twitter-sentimental-analysis/0000000000002747_test_twitter_x_test.csv')
dft

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...
3655,570304244001193984,US Airways,NaN,Anthony_Scerri,NaN,0,@USAirways Been stuck for 40+ minutes due to l...,NaN,2015-02-24 11:28:22 -0800,"Astoria, NY",Quito
3656,567847737061941249,US Airways,NaN,mttdprkr,NaN,0,@USAirways 4 hours... 4 hours... FOUR HOURS. ...,NaN,2015-02-17 16:47:05 -0800,"Vancouver, WA",Pacific Time (US & Canada)
3657,567823564167192576,Virgin America,NaN,miaerolinea,NaN,1,Nice RT @VirginAmerica: The man of steel might...,NaN,2015-02-17 15:11:02 -0800,Worldwide,Caracas
3658,570273819287531520,American,NaN,GoldensPleasure,NaN,0,@AmericanAir Aww Thanks AA..DFW was on GMA up ...,NaN,2015-02-24 09:27:28 -0800,East Coast CT.,Central Time (US & Canada)


In [33]:
dft.drop(['tweet_id','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','user_timezone','tweet_location','tweet_created'],axis=1,inplace=True)
dft

,text
0,@AmericanAir In car gng to DFW. Pulled over 1h...
1,"@AmericanAir after all, the plane didn’t land ..."
2,@SouthwestAir can't believe how many paying cu...
3,@USAirways I can legitimately say that I would...
4,@AmericanAir still no response from AA. great ...
...,...
3655,@USAirways Been stuck for 40+ minutes due to l...
3656,@USAirways 4 hours... 4 hours... FOUR HOURS. ...
3657,Nice RT @VirginAmerica: The man of steel might...
3658,@AmericanAir Aww Thanks AA..DFW was on GMA up ...


In [34]:
dft['clean_test_text'] = np.vectorize(remove_pattern)(dft['text'], "@[\w]*")
dft

,text,clean_test_text
0,@AmericanAir In car gng to DFW. Pulled over 1h...,In car gng to DFW. Pulled over 1hr ago - very...
1,"@AmericanAir after all, the plane didn’t land ...","after all, the plane didn’t land in identical..."
2,@SouthwestAir can't believe how many paying cu...,can't believe how many paying customers you l...
3,@USAirways I can legitimately say that I would...,I can legitimately say that I would have rath...
4,@AmericanAir still no response from AA. great ...,still no response from AA. great job guys!
...,...,...
3655,@USAirways Been stuck for 40+ minutes due to l...,Been stuck for 40+ minutes due to lavatory is...
3656,@USAirways 4 hours... 4 hours... FOUR HOURS. ...,4 hours... 4 hours... FOUR HOURS. It's like ...
3657,Nice RT @VirginAmerica: The man of steel might...,"Nice RT : The man of steel might be faster, bu..."
3658,@AmericanAir Aww Thanks AA..DFW was on GMA up ...,Aww Thanks AA..DFW was on GMA up here this AM...


In [35]:
test_Documents = [word_tokenize(doc) for doc in list(dft['clean_test_text'])]
test_Documents

[['In',
  'car',
  'gng',
  'to',
  'DFW',
  '.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads',
  '.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr',
  '.',
  'Ca',
  "n't",
  'reach',
  'arpt',
  'for',
  'AA2450',
  '.',
  'Wat',
  '2',
  'do',
  '?'],
 ['after',
  'all',
  ',',
  'the',
  'plane',
  'didn',
  '’',
  't',
  'land',
  'in',
  'identical',
  'or',
  'worse',
  ')',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs',
  '.'],
 ['ca',
  "n't",
  'believe',
  'how',
  'many',
  'paying',
  'customers',
  'you',
  'left',
  'high',
  'and',
  'dry',
  'with',
  'no',
  'reason',
  'for',
  'flight',
  'Cancelled',
  'Flightlations',
  'Monday',
  'out',
  'of',
  'BDL',
  '!',
  'Wow',
  '.'],
 ['I',
  'can',
  'legitimately',
  'say',
  'that',
  'I',
  'would',
  'have',
  'rather',
  'driven',
  'cross',
  'country',
  'than',
  'flown',
  'on',
  'US',
  'Airways',
  '.'],
 ['still', 'no', 'response', 'from', 'AA', '.',

In [36]:
all_test_Documents=[]
#this variable is going to contain all the words from all our tokenized documents.
for doc in test_Documents:
    list_of = []
    for word in doc:
        if (word.lower() not in stops) and len(word)!=1 and len(word)!=2 and word[0]!="'" and word!="n't" and word[0]!=".":
            #I dont't want to include words with length 1 and 2 in my vocabulary because these words are pretty much useless.
            # and they might either be the stops which are not there in "stops" variable or some punctuation marks which are
            # not there in "punctuations" variable. apart from the words of length 1 and 2 i have also removed some words separately because these words are most probably present in all the documents so they are not helping us much in classification.
            list_of.append(word.lower())
    all_test_Documents.append(list_of)
all_test_Documents

[['car',
  'gng',
  'dfw',
  'pulled',
  '1hr',
  'ago',
  'icy',
  'roads',
  'on-hold',
  'since',
  '1hr',
  'reach',
  'arpt',
  'aa2450',
  'wat'],
 ['plane',
  'land',
  'identical',
  'worse',
  'conditions',
  'grk',
  'according',
  'metars'],
 ['believe',
  'many',
  'paying',
  'customers',
  'left',
  'high',
  'dry',
  'reason',
  'flight',
  'cancelled',
  'flightlations',
  'monday',
  'bdl',
  'wow'],
 ['legitimately',
  'say',
  'would',
  'rather',
  'driven',
  'cross',
  'country',
  'flown',
  'airways'],
 ['still', 'response', 'great', 'job', 'guys'],
 ['developers',
  'flying',
  'tmrw',
  'morn',
  'w/45',
  'min',
  'layover',
  'earlier',
  'flight',
  '1.5hr',
  'layover',
  'move'],
 ['hello', 'anyone'],
 ['mr.',
  'husain',
  'shld',
  'protest',
  'well',
  'one',
  'party',
  'member',
  'rehman',
  'malik',
  'delayed',
  'pia',
  'flight',
  'hours'],
 ['likely', 'flightaware', 'says', 'plane', 'still', 'durango', 'departed'],
 ['even',
  'give',
  'opt

In [37]:
testDocuments = []
for i in range(3660):
    testDocuments.append(([stemmer.stem(x) for x in all_test_Documents[i]]))
testDocuments

[['car',
  'gng',
  'dfw',
  'pull',
  '1hr',
  'ago',
  'ici',
  'road',
  'on-hold',
  'sinc',
  '1hr',
  'reach',
  'arpt',
  'aa2450',
  'wat'],
 ['plane', 'land', 'ident', 'wors', 'condit', 'grk', 'accord', 'metar'],
 ['believ',
  'mani',
  'pay',
  'custom',
  'left',
  'high',
  'dri',
  'reason',
  'flight',
  'cancel',
  'flightlat',
  'monday',
  'bdl',
  'wow'],
 ['legitim',
  'say',
  'would',
  'rather',
  'driven',
  'cross',
  'countri',
  'flown',
  'airway'],
 ['still', 'respons', 'great', 'job', 'guy'],
 ['develop',
  'fli',
  'tmrw',
  'morn',
  'w/45',
  'min',
  'layov',
  'earlier',
  'flight',
  '1.5hr',
  'layov',
  'move'],
 ['hello', 'anyon'],
 ['mr.',
  'husain',
  'shld',
  'protest',
  'well',
  'one',
  'parti',
  'member',
  'rehman',
  'malik',
  'delay',
  'pia',
  'flight',
  'hour'],
 ['like', 'flightawar', 'say', 'plane', 'still', 'durango', 'depart'],
 ['even',
  'give',
  'option',
  'hold',
  'say',
  'line',
  'busi',
  'plz',
  'tri',
  'late',


In [38]:
test_text_documents = [" ".join(document) for document in testDocuments]
test_text_documents

['car gng dfw pull 1hr ago ici road on-hold sinc 1hr reach arpt aa2450 wat',
 'plane land ident wors condit grk accord metar',
 'believ mani pay custom left high dri reason flight cancel flightlat monday bdl wow',
 'legitim say would rather driven cross countri flown airway',
 'still respons great job guy',
 'develop fli tmrw morn w/45 min layov earlier flight 1.5hr layov move',
 'hello anyon',
 'mr. husain shld protest well one parti member rehman malik delay pia flight hour',
 'like flightawar say plane still durango depart',
 'even give option hold say line busi plz tri late flightr',
 'announc pre board address mobil disabl requir travel lot stuff preboard',
 'realli embarrass ask complimentari drink/snack detail http //t.co/9za6xb1h89 amp argu',
 'passport time trip could still fli photo thingsishouldknow ifeeldumb',
 'delay bag friend lisa pafe got bag day costa rica issu updat system',
 'see travel compet unus fund expir date hidden fine print never saw',
 'awesom door close min

In [39]:
x_test_features = count_vec.transform(test_text_documents)
x_test_features

<3660x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 30137 stored elements in Compressed Sparse Row format>

In [40]:
x_test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [41]:
y_pred = svc.predict(x_test_features)
y_pred

array([0, 1, 0, ..., 0, 2, 0])

In [42]:
len(y_pred)
unformated_result = list(y_pred)
unformated_result

[0,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 0,


In [43]:
result = []
for i in unformated_result:
    if i == 0:
        result.append('negative')
    elif i == 1:
        result.append('neutral')
    elif i == 2:
        result.append('positive')
result

['negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'positive',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negati

In [44]:
np.savetxt("Twitter_Sentiment_Analysis_predictions.csv",result,fmt="%s")

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
clf = RandomForestClassifier()

In [47]:
clf.fit(x_train_features, categories)

RandomForestClassifier()

In [48]:
clf.score(x_train_features, categories)

0.9894353369763206

In [49]:
y_pred = clf.predict(x_test_features)
y_pred

array([0, 1, 0, ..., 1, 2, 0])

In [50]:
len(y_pred)
unformated_result = list(y_pred)
unformated_result

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,


In [51]:
result = []
for i in unformated_result:
    if i == 0:
        result.append('negative')
    elif i == 1:
        result.append('neutral')
    elif i == 2:
        result.append('positive')
result

['negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negati

In [52]:
np.savetxt("Twitter_Sentiment_Analysis_predictions.csv",result,fmt="%s")